In [1]:
from PIL import ImageColor, Image, ImageDraw, ImageFilter


In [2]:
print(ImageColor.getcolor('red', 'RGBA'))


(255, 0, 0, 255)


In [6]:
img = Image.new("RGB", (500, 500), (173, 216, 230))
img.show()

In [30]:
img = Image.new("RGBA", (500, 500), (255, 0, 0, 32))
img.show()
img.save('./a32.jpg')
img.save('./a32.gif')
img.save('./a32.png') # has the alpha information (opaque)

In [41]:
img = Image.open('lena.jpg')
img.mode
# Lets resize the input image by x3
x, y = img.size
mag = 3
newsize = (mag*x, mag*y)
img.resize(newsize).show()

# usage: thumbnail(size, resample=3) requested size, resample = OPTIONAL BILINEAR, BICUBIC, NEAREST
# the method modifies the image itself (watch out for artifacts)
# Here is a more Pythonic way to do this.
newsize = tuple(size/2 for size in img.size)
img.thumbnail(newsize) 
img.save('thumb.jpeg')

newsize = tuple(size*mag for size in img.size)
img.resize(newsize).show()

x1 = 300
y1 = 20
x2 = 800
y2 = 500
img1 = img.resize(newsize).crop((x1, y1, x2, y2)).rotate(-45)
img1.show()
img1.save('./r.jpg')
img1.save('./r10.jpg', quality=5)
img1.save('./r50.jpg', quality=10)
img1.save('./r50.jpg', quality=50)
img1.save('./r100.jpg', quality=100)


In [33]:
img = Image.open('lena.jpg')
# reading a pixel data
img.getpixel((0,0)) 
# There are 2 ways obtain the pixel values in the image (read/write)
# 1. getpixel()/putpixel()
w, h = img.size
for x in range(h):
    img.putpixel((x, x), (0, 0, 0, 255))

# 2. using a PixelAccess obj to easy access []
imgpx = img.load() #Load image data based on sequence of data in a list [x,y]
imgpx[0, 0]
# you can also write the values to the image
for x in range(h):
    imgpx[x, 100] = (255, 0, 0)
img.show()


In [35]:
# updating a jpg and adding opaque then save it into PNG format
img = Image.open('lena.jpg')
ima = img.copy()
ima.show()
img.putalpha(128)
img.show()
img.save('./lena_a.png')

In [37]:
# -- let's now composite 2 images with transparency control
# let's look at the composite method
# Image.composite(i1, i2, msk): Create composite image by blending images using a transparency mask.
# image 1 + image 2 + mask image(-> blending weight)
im1 = Image.open('./megryan.jpg')                  # base image
im2 = Image.open('./seattle.jpg').resize(im1.size) # new image

import numpy
# creates a new image by interpolating between two input images, using a constant alpha.
Image.blend(im1, im2, 0.5)
# out = image1 * (1.0 - alpha) + image2 * alpha

for i in numpy.arange(0, 1.2, 0.2):
    img = Image.blend(im1, im2, i)
    img.show()

# create a mask with grayscale image
mask = Image.new("L", im1.size, 128) # 'L'=8-bit pixels, black and white
imcomp = Image.composite(im1, im2, mask) # mask = weight


In [38]:

# -- Let's add look at a shape
mask = Image.new("L", im1.size, 0) # 
# The ImageDraw module provides simple 2D graphics for Image objects. 
# You can use this module to create new images, annotate or retouch 
# existing images, and to generate graphics on the fly for web use.
# https://pillow.readthedocs.io/en/stable/reference/ImageDraw.html
draw = ImageDraw.Draw(mask)
draw.ellipse((180, 26, 320, 173), fill=255) # [(x0, y0, x1, y1]
mask.show() # show the mask image

# composite thru mask image (weight factor per pixel)
im = Image.composite(im1, im2, mask) # im = im1 * mask + im2

# In order to make the border smooth, we can use some of the ImageFilter module
mask_blur = mask.filter(ImageFilter.GaussianBlur(20))
mask_blur.show()
im = Image.composite(im1, im2, mask_blur)
im.show()



In [43]:
# - Let's now crop and create a sub-image
#   And save to a different image format 
im = Image.open('zophie.png') 
# a quick way to see an image on the disk Image.open('zophie.png').show()
im.format
im.format_description
print(im.format, im.width, im.height, im.size, im.mode)
box = (340,338,568,579)
# Verify the cropped image
im.crop(box).show()
im_crop = im.crop(box)
# Save it to different formats
im_crop.save('cropped.png')
im_crop.save('cropped.ppm')
im_crop.save('cropped.bmp')
im_crop.save('cropped.gif')


PNG 816 1088 (816, 1088) RGB


In [42]:
# crop out and create a subimage, then paste into a certain location
catIm = Image.open('zophie.png')
catCopyIm = catIm.copy()
box = (340,338,568,579)
catFaceIm = catCopyIm.crop(box)
# target.paste
catCopyIm.paste(catFaceIm, (0, 0))
# at this point, catCopyIm has pasted face image
catCopyIm.show()


In [49]:

# Making a tile image and rotation
catIm = Image.open('zophie.png')
box = (360,338,523,579) # 360 + (816/5=163) = 523
catFaceIm = catIm.crop(box)
catImWidth, catImHeight = catIm.size
catFaceImWidth, catFaceImHeight = catFaceIm.size
# Since this paste operation will modify the target image, need a copy.
catCopyImTile = catIm.copy()
i = 0
for x in range(0, catImWidth, catFaceImWidth):
    for y in range(0, catImHeight, catFaceImHeight):
        print(x, y)
        catCopyImTile.paste(catFaceIm, (x, y))
        catCopyImTile.paste(catFaceIm.rotate(10*i), (x, y))
        i += 1
catCopyImTile.show()    


0 0
0 241
0 482
0 723
0 964
163 0
163 241
163 482
163 723
163 964
326 0
326 241
326 482
326 723
326 964
489 0
489 241
489 482
489 723
489 964
652 0
652 241
652 482
652 723
652 964
815 0
815 241
815 482
815 723
815 964


In [53]:

def create_image_with_ball(width, height, ball_x, ball_y, ball_size):
    # create a white background
    img = Image.new('RGB', (width, height), (255, 255, 255))
    draw = ImageDraw.Draw(img)
    # draw.ellipse takes a 4-tuple (x0, y0, x1, y1) where (x0, y0) is the top-left bound of the box
    # and (x1, y1) is the lower-right bound of the box.
    draw.ellipse((ball_x, ball_y, ball_x + ball_size, ball_y + ball_size), fill='red')
    return img

# Create the frames
frames = []
x, y = 0, 0
for i in range(20):
    new_frame = create_image_with_ball(400, 400, x, y, 80)
    # new_frame = create_image_with_ball(400, 400, x, y, 400/(i+1)) # 3D effect going away from me trajectory
    frames.append(new_frame)
    x += 20
    y += 20

# Save into a GIF file that loops forever
frames[0].save('moving.gif', format='GIF', append_images=frames[1:], save_all=True, duration=100)

In [61]:
# Let's make an animated gif file.
# Imagine that you have a field of view from the camera (200 X 200), and it scans thru 
# a diagonal path, and display them on as an animated gif.
catIm = Image.open('zophie.png')
box = (0, 0, 200, 200)

# Create the frames
frames = []
x, y = 0, 0
for i in range(0, catIm.width-200, 10):
    new_frame = catIm.crop(box) 
    frames.append(new_frame)
    box = tuple(x + 10 for x in box)
# Save into a GIF file that loops forever
frames[0].save('moving_cat.gif', format='GIF', append_images=frames[1:], save_all=True, duration=100)


In [ ]:
# 1. open an image
# 2. crop the cat's face using box tuple (x0, y0, x1, y1) with crop() then store it into another Image object
# 3. create the frames starting with empty frame
# 4. rotate 360 by 1 degree using range() followed by (1) rotate() and (2) append the frames
# 5. save the frames using format=GIF, append_images, save_all, duration to control the frame rate


In [65]:
# Let's make an animated gif file.
# Imagine that you have a field of view from the camera (200 X 200), and it scans thru 
# a diagonal path, and display them on as an animated gif.
catIm = Image.open('zophie.png')
box = (360,338,523,579) # 360 + (816/5=163) = 523
catFaceIm = catIm.crop(box)

# Create the frames
frames = []
for i in range(0, 360):
    new_frame = catFaceIm.rotate(i) 
    frames.append(new_frame)
    
# Save into a GIF file that loops forever
frames[0].save('rotating_cat.gif', format='GIF', append_images=frames[1:], save_all=True, duration=10)